Análisis REM8
 ==========

Comienzo cargando todos los paquetes necesarios

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy 
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import datetime as dt
import datetime
from datetime import timedelta  
import timeboard as tb # turnos 

# para tratar de poner espanol
from datetime import date, datetime, time
from babel.dates import format_date, format_datetime, format_time, format_timedelta, Locale
import locale                                    # para tratar de poner espanol
#locale = Locale('es', 'CL')
#locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')  

import os
import runpy
from pyexcel_ods import get_data
from matplotlib_venn import venn3
from matplotlib_venn import venn2, venn2_circles
import math

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline


#otro graficador
import seaborn as sns

#stats

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# cargo algunos scripts que me suelen servir para análisis

In [3]:
#listo los archivos
scripts = []
for dirname, dirnames, filenames in os.walk('/home/egidio/Dropbox/Compucosas/programas py'):
    for filename in filenames:
        scripts.append(os.path.join(dirname, filename))

scripts

# los cargo
for script in scripts:
    if script[-3:] == '.py':
        exec(open(str(script)).read())


['/home/egidio/Dropbox/Compucosas/programas py/percentiles.py',
 '/home/egidio/Dropbox/Compucosas/programas py/características_operativas_de_test.py',
 '/home/egidio/Dropbox/Compucosas/programas py/lector_ODF.py',
 '/home/egidio/Dropbox/Compucosas/programas py/como importar',
 '/home/egidio/Dropbox/Compucosas/programas py/tabala_frecuencias.py',
 '/home/egidio/Dropbox/Compucosas/programas py/descripcion_univariada.py',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/características_operativas_de_test.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/tabala_frecuencias.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/lector_ODF.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/percentiles.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/funciones_locales.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/descripcion_univariada.cpython-36.pyc']

# Cargo la BD

In [4]:
BD = pd.read_pickle('./BD/eSAMU.pkl')

In [5]:
BD.columns
BD.Fecha.describe()
BD.head(2)


Index(['Id', 'Fecha', 'Motivo del Llamado', 'Submotivo del Llamado',
       'Prioridad', 'Estado', 'Edad', 'Género', 'Dirección', 'Sector',
       'Comuna', 'Región', 'Intervención', 'Tipo Traslado',
       'Categoría Vehículo', 'Condición Traslado', 'En Regulación',
       'En Despacho', 'En Curso', 'Atendida', 'Completado', 'Regulado',
       'Usuario que Cierra', 'Móvil Agregado', 'Móvil Despachado',
       'Móvil Salida', 'Móvil Llegada Lugar', 'Móvil en Traslado',
       'Móvil Llegada a CA', 'Móvil Recepción Paciente',
       'Móvil Camilla Retenida', 'Móvil Vuelta a Base', 'Móvil LLegada a Base',
       'Móvil Finalizado', 'Nombre Vehículo', 'Tipo Requerido',
       'Tipo Despachado', 'Km Recorrido', 'Chofer', 'Paramédico', 'Enfermero',
       'Doctor', 'Longitud', 'Latitud', 'Diff_recep_Despachado',
       'Diff_Despacho_Salida', 'Diff_salida_enellugar', 'Diff_z6_1',
       'Diff_z6_2', 'Diff_z6_3'],
      dtype='object')

count                  129810
unique                 114230
top       2018-11-23 14:30:10
freq                      442
first     2018-01-15 08:45:15
last      2020-06-30 23:57:16
Name: Fecha, dtype: object

,Id,Fecha,Motivo del Llamado,Submotivo del Llamado,Prioridad,Estado,Edad,Género,Dirección,Sector,...,Enfermero,Doctor,Longitud,Latitud,Diff_recep_Despachado,Diff_Despacho_Salida,Diff_salida_enellugar,Diff_z6_1,Diff_z6_2,Diff_z6_3
0,144550,2020-03-31 23:53:04,Dolor o Problema específico,Dificultad respiratoria leve - moderada (disnea),NaN,Completado,53.0,Masculino,x calle del agua pje 10 casa 161 b,recreo,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT
1,144549,2020-03-31 23:42:58,Dolor o Problema específico,Dolor o problemas cabeza (cefalea),NaN,Regulado,34.0,Femenino,calle anibal pinto 45 dpto 305,"c12 , reñaca bajo",...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT


para efectos de este análisis es importante dividir ambulancias según su complejidad

In [6]:
BD['Bas_avan'] = BD['Tipo Despachado'] 
BD.loc[BD['Tipo Despachado'] == 'm1' , 'Bas_avan' ] = 'Básico'
BD.loc[BD['Tipo Despachado'] == 'm2' , 'Bas_avan' ] = 'Avanzado'
BD.loc[BD['Tipo Despachado'] == 'm3' , 'Bas_avan' ] = 'Avanzado'
BD.loc[BD['Tipo Despachado'] == 'x5' , 'Bas_avan' ] = 'Avanzado'

defino Bas_avan como categórico

# defino la BD con la que trabajaré

El informe a continuación es un análisis del:
* fecha
* sector

## según fecha

In [7]:
rango = (                                            #mes Junio
    (BD.Fecha > pd.to_datetime('2020-06-01')) &
    (BD.Fecha < pd.to_datetime('2020-06-30'))
)

db1 = BD[rango].copy()
np.sort(db1.Fecha.dt.to_period('M').unique())

array([Period('2020-06', 'M')], dtype=object)

## segun ambulancias

In [8]:
db1['Nombre Vehículo'].unique()

array(['R32', nan, 'R33', 'R42', 'R63', 'R31', 'R83', 'R73', 'R26', 'R12',
       'AP16', 'AP15', 'R23', 'R92', 'R80', 'R21', 'R64', 'R10', 'R62',
       'R24', 'R41', 'AP31', 'R61', 'R22', 'R50', 'AP 14', 'AP51', 'R11',
       'AP1', 'R91', 'R25', 'R13', 'AT90', 'AP28', 'R43', 'R51', 'AT51',
       'Móvil Las Parcelas', 'R101', 'AP41', 'R100', 'R71', 'R52', 'R72',
       'AP-17 Loncura', 'W56', 'R53', 'R34', 'Móvil Hijuelas', 'R57',
       'W133 Carabineros', 'AP27', 'Móvil Ventana', 'Móvil Artificio',
       'Móvil Puchuncaví', 'Móvil Nogales', 'MOVIL LONCURA',
       'sapu zapallar', 'R84', 'AP26', 'MOVIL SAPU VENTANA', 'R82', 'R20',
       'R44', 'Corona-movil petorca', 'REMA  Poli Naval Villa Alemana',
       'MOVIL RUTA'], dtype=object)

elegir uno

In [9]:
# Comentar todo o no

sector = db1['Nombre Vehículo'].str.contains('R3') == True
db1 = db1[sector]

In [10]:
texto1 = 'Este es un inforque que considera el mes de '
texto2 = 'En el territorio de '

# sector = 'SSVQ'
sector = db1.Comuna.value_counts().index[0]

mes= db1.Fecha.describe()[4].strftime('%B')

result = texto1 + mes + '. ' + texto2 + sector + '.' + '\n'
result
print(result, file=open('Informe.txt','a'))

'Este es un inforque que considera el mes de June. En el territorio de Quilpué.\n'

In [11]:
db1.Comuna.value_counts()

Quilpué          316
Villa Alemana     14
Name: Comuna, dtype: int64

# defino el archivo donde guardaré la info

Será un archivo

como dividir por fecha

In [12]:
np.sort(db1.Fecha.dt.to_period('M').unique())

array([Period('2020-06', 'M')], dtype=object)

## Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)

prestaciones según recurso

In [13]:
# los tiempos de z17 al z8 los divido según requisito planilla
cortado = pd.cut(
    db1.Diff_salida_enellugar.dt.seconds,
    [0, 20, 40, 1000000],
    labels = ['Menos 20 min',  '20 a 40 min',  'Más 40 min'])

# tabulo
#pd.crosstab(db1.Bas_avan, cortado, margins=True, margins_name= 'Total')

# tabulo por mes
print('Por Mes')
dum1 = cortado
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    result = pd.crosstab(
        test['Bas_avan'],
        dum1,
        margins=True, margins_name= 'Total', dropna=False)
    result.columns.name = 'Tiempo llegada al lugar'
    result.index.name = 'Tipo Móvil'
    result

Por Mes


Period('2020-06', 'M')

Tiempo llegada al lugar,Menos 20 min,20 a 40 min,Más 40 min,Total
Tipo Móvil,,,,
Avanzado,0,3,77,119
Básico,2,0,164,283
Total,2,3,302,502


escribo en el archivo

In [14]:
titulo = 'Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)'
texto = 'Se asume que todas las intervenciones críticas son realizadas \
por móviles avanzados y que las no-críticas por móviles básicos. Esto \
determina que las casillas de las prestaciones críticas tengan valores \
de cero para móviles básicos y las prestaciones no-críticas tengan \
valores cero para móviles avanzados. \n \n'
espacios = '\n \n'

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

'Sección K: INTERVENCIONES PRE HOSPITALARIAS (SAMU)'

'Se asume que todas las intervenciones críticas son realizadas por móviles avanzados y que las no-críticas por móviles básicos. Esto determina que las casillas de las prestaciones críticas tengan valores de cero para móviles básicos y las prestaciones no-críticas tengan valores cero para móviles avanzados. \n \n'

Tiempo llegada al lugar,Menos 20 min,20 a 40 min,Más 40 min,Total
Tipo Móvil,,,,
Avanzado,0,3,77,119
Básico,2,0,164,283
Total,2,3,302,502


## Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA (Desde el lugar del evento a unidad de Emergencia)

In [15]:
db1['Categoría Vehículo'].value_counts()
#pd.crosstab(db1['Categoría Vehículo'], test['Bas_avan'])

db1['Categoría Vehículo'] = pd.Categorical(db1['Categoría Vehículo'],
                                           categories=['samu', 'enrutado', 'nosamu'])

enrutado    316
Name: Categoría Vehículo, dtype: int64

In [16]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    test = test.loc[test['Motivo del Llamado'] != 'Traslados']
    result = test.groupby(['Categoría Vehículo', 'Bas_avan' ])['Id'].count()
    result.index.names = ['Categoría Vehículo', 'Tipo móvil']
    result

Period('2020-06', 'M')

Categoría Vehículo  Tipo móvil
samu                Avanzado       0
                    Básico         0
enrutado            Avanzado      54
                    Básico        99
nosamu              Avanzado       0
                    Básico         0
Name: Id, dtype: int64

escribo en el archivo

In [17]:
titulo = ' Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA'
texto = ' '

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

' Sección L:  TRASLADOS PRIMARIOS A UNIDADES DE URGENCIA'

' '

Categoría Vehículo  Tipo móvil
samu                Avanzado       0
                    Básico         0
enrutado            Avanzado      54
                    Básico        99
nosamu              Avanzado       0
                    Básico         0
Name: Id, dtype: int64

## Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)

In [18]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    test = test.loc[test['Motivo del Llamado'] == 'Traslados']
    result = test.groupby(['Categoría Vehículo', 'Bas_avan' ])['Id'].count()
    result.index.names = ['Categoría Vehículo', 'Tipo móvil']
    result

Period('2020-06', 'M')

Categoría Vehículo  Tipo móvil
samu                Avanzado       0
                    Básico         0
enrutado            Avanzado      27
                    Básico        74
nosamu              Avanzado       0
                    Básico         0
Name: Id, dtype: int64

In [19]:
titulo = ' Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'
texto = ' '

titulo
texto
result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result, file=open('Informe.txt','a'))
print(espacios, file=open('Informe.txt','a'))

' Sección M: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'

' '

Categoría Vehículo  Tipo móvil
samu                Avanzado       0
                    Básico         0
enrutado            Avanzado      27
                    Básico        74
nosamu              Avanzado       0
                    Básico         0
Name: Id, dtype: int64

# Sección N: CLASIFICACION DE LAS INTERVENCIONES POR GRANDES GRUPOS DE DIAGNOSTICOS (SAMU)

In [20]:
db1['Patol'] = np.nan
db1.loc[db1['Submotivo del Llamado'] == 'Accidente vehicular o transporte. Atropellos',
        'Patol'] = 'PoliTMT'
db1.loc[db1['Submotivo del Llamado'] == 'Accidente múltiples víctimas (cualquier razón)',
        'Patol'] = 'PoliTMT'
db1.loc[db1['Submotivo del Llamado'] == 'Caídas graves',
        'Patol'] = 'PoliTMT'

db1.loc[db1['Submotivo del Llamado'] == 'Colapso respiratorio o circulatorio. PCR. Asfixia',
        'Patol'] = 'PCR'

db1.loc[db1['Submotivo del Llamado'] == 'Dolor de pecho',
        'Patol'] = 'SCA'

db1.loc[db1['Patol'].isnull(),
        'Patol'] = 'Otros'

db1['Patol'] = pd.Categorical(db1['Patol'], categories=['SCA', 'PCR', 'PoliTMT', 'Otros'])

In [21]:
db1.loc[db1['Género'] == 'Femenino', 'Género'] = 'Feme'
db1.loc[db1['Género'] == 'Masculino', 'Género'] = 'Masc'

db1['Género'] = pd.Categorical(db1['Género'], categories=['Masc', 'Feme'])

In [22]:
for fecha in np.sort(db1.Fecha.dt.to_period('M').unique()):
    fecha
    test = db1.loc[db1.Fecha.dt.to_period('M') == fecha]
    dum1 = pd.cut(test.Edad, range(0, 90, 5), right=False)  
    test['Patol'].value_counts(sort= False)
    result0 = pd.crosstab(test['Patol'], test['Género'])
    separador = ' -----------------------------------------------------------------'
    result1 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,:8]
    result2 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,8:16]
    result3 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,16:24]
    result4 = pd.crosstab(test['Patol'], [dum1, test['Género']]).iloc[:,24:]
    result0
    result1
    result2
    result3
    result4

Period('2020-06', 'M')

SCA         11
PCR          4
PoliTMT     41
Otros      446
Name: Patol, dtype: int64

Género,Masc,Feme
Patol,,
SCA,4,6
PCR,2,2
PoliTMT,26,14
Otros,220,216


Edad    [0, 5)      [5, 10)      [10, 15)      [15, 20)     
Género    Masc Feme    Masc Feme     Masc Feme     Masc Feme
Patol                                                       
SCA          0    0       0    0        0    0        0    0
PCR          0    0       0    0        0    0        0    0
PoliTMT      0    0       0    0        0    2        2    1
Otros        4    5       9    3        4    3        7    7

Edad    [20, 25)      [25, 30)      [30, 35)      [35, 40)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                          
SCA            0    0        0    0        0    0        0    0
PCR            0    0        0    0        0    0        0    0
PoliTMT        2    1        1    1        5    1        3    1
Otros          8    7        8    7        7    1        1    4

Edad    [40, 45)      [45, 50)      [50, 55)      [55, 60)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                          
SCA            0    1        2    0        1    0        0    1
PCR            0    0        1    0        0    0        0    0
PoliTMT        3    0        2    1        2    1        1    1
Otros          9   18       10    8       11    9       11    9

Edad    [60, 65)      [65, 70)      [70, 75)      [75, 80)      [80, 85)     
Género      Masc Feme     Masc Feme     Masc Feme     Masc Feme     Masc Feme
Patol                                                                        
SCA            0    0        0    2        1    0        0    1        0    1
PCR            0    1        0    1        1    0        0    0        0    0
PoliTMT        0    2        2    0        1    0        0    0        0    1
Otros         26   14       13   13       34   15       14   21       20   21

In [23]:
titulo = ' Sección N: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'
texto = 'Es una tabla larga, por lo que se divide en varios espacios '

titulo
texto
#result

print(titulo, file=open('Informe.txt','a'))
print(texto, file=open('Informe.txt','a'))
print(result0, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result1, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result2, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result3, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(result4, file=open('Informe.txt','a'))
print(separador,file=open('Informe.txt','a'))  #Separador
print(espacios, file=open('Informe.txt','a'))

' Sección N: TRASLADO SECUNDARIO (Desde un Establecimiento a Otro)'

'Es una tabla larga, por lo que se divide en varios espacios '

# Finalización
Fin informe REM8

In [24]:
print('Fin informe REM 8', file=open('Informe.txt','a'))


# Análisis posteriores extra

In [25]:
db1[db1['Motivo del Llamado']=='Otro'][['Submotivo del Llamado', 'Id']]

,Submotivo del Llamado,Id
412,Fallecido,165088
466,Fallecido,165022
490,Fallecido,164997
572,Fallecido,164907
645,Fallecido,164815
985,Fallecido,164318
1025,Fallecido,164257
1192,Fallecido,164024
1264,Fallecido,163943
1280,Fallecido,163925


In [26]:
BD.Fecha.describe()

count                  129810
unique                 114230
top       2018-11-23 14:30:10
freq                      442
first     2018-01-15 08:45:15
last      2020-06-30 23:57:16
Name: Fecha, dtype: object